In [28]:
%pip install chargebee odooly pandas
%pip install google-cloud-bigquery
%pip install boto3

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 KB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 KB 4.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━

In [48]:
import chargebee
import odooly
import pandas as pd
import json
import xmlrpc.client
from google.cloud import bigquery
from google.oauth2 import service_account
import boto3
from typing import Union

In [50]:
def fetch_ssm_credentials(
    credentials: str,
    type: str = "dict",
    region: str = "us-east-1",
    with_decryption: bool = True,
) -> Union[str, dict]:
    """
    This function fetches a parameter from AWS SSM Parameter Store and returns it

    :param credentials: The name of the parameter in SSM
    :type credentials: str
    :param type: str = "dict", defaults to dict
    :type type: str (optional)
    :param region: The AWS region where the SSM parameter is stored, defaults to us-east-1
    :type region: str (optional)
    :param with_decryption: If the parameter is encrypted, this will decrypt it, defaults to True
    :type with_decryption: bool (optional)
    """
    aws_session = boto3.Session()
    ssm = aws_session.client("ssm", region_name=region)
    response = ssm.get_parameter(Name=credentials, WithDecryption=with_decryption)
    if type == "str":
        return response["Parameter"]["Value"]
    return json.loads(response["Parameter"]["Value"])



TypeError: fetch_ssm_credentials() missing 1 required positional argument: 'credentials'

In [14]:
# Configuración de Chargebee
chargebee.configure('test_UIH0hwBMGEgXrq7isGWyGxo4yW4cY1Qr', 'cliengo-test')

In [37]:
# Configuración de Odoo
 
import requests

url = "https://demo-28-08-6.adhoc.ar/jsonrpc"
db = "demo-28-08-6"
username = "Chargebee"
password = "Chargebee"
headers = {'Content-Type': 'application/json'}

# Preparar conexión
data = {
    "jsonrpc": "2.0",
    "method": "call",
    "params": {
        "service": "common",
        "method": "login",
        "args": [db, username, password]
    },
    "id": 1
}

# Realizar solicitud HTTP POST
response = requests.post(url, headers=headers, json=data)
result = response.json()

if 'result' in result:
    uid = result['result']
    print(f"Logged in with UID: {uid}")
else:
    print("Failed to log in")
    exit(1)



Logged in with UID: 15


In [47]:

# Busco Invoices en Odoo
query_data = {
    "jsonrpc": "2.0",
    "method": "call",
    "params": {
        "service": "object",
        "method": "execute_kw",
        "args": [db, uid, password, 'sale.order', 'search_read', [[['id', '>', 1]], 0, 10]],
    },
    "id": 2
}

query_response = requests.post(url, headers=headers, json=query_data)
#print(query_response)
result = query_response.json()
#print(result)

for invoice in result['result']:
    print(invoice)

#print(result['result'])

{'id': 329, 'campaign_id': False, 'source_id': False, 'medium_id': False, 'activity_ids': [], 'activity_state': False, 'activity_user_id': False, 'activity_type_id': False, 'activity_type_icon': False, 'activity_date_deadline': False, 'my_activity_date_deadline': False, 'activity_summary': False, 'activity_exception_decoration': False, 'activity_exception_icon': False, 'activity_calendar_event_id': False, 'message_is_follower': False, 'message_follower_ids': [1146], 'message_partner_ids': [7], 'message_ids': [2468], 'has_message': True, 'message_needaction': False, 'message_needaction_counter': 0, 'message_has_error': False, 'message_has_error_counter': 0, 'message_attachment_count': 0, 'rating_ids': [], 'website_message_ids': [], 'message_has_sms_error': False, 'access_url': '/my/orders/329', 'access_token': False, 'access_warning': '', 'name': 'V 0001-00000329', 'company_id': [1, 'Muebleria US'], 'partner_id': [11, 'Gemini Furniture'], 'state': 'draft', 'locked': False, 'client_order

In [19]:
# me conecto a chargebee para hacer una primera lectura de prueba de invoices

def get_chargebee_invoices():
    entries = chargebee.Invoice.list({
        "limit" : 5,
        "status[in]" : ["paid","payment_due"],
        "sort_by[asc]" : "date"
        })
    for entry in entries:
        invoice = entry.invoice
        print(invoice)
    
get_chargebee_invoices()

{
    "id": "46794",
    "customer_id": "5dde8435e4b098a27bbd4def",
    "subscription_id": "AzyWD9Rj28XRP3nc",
    "recurring": true,
    "status": "paid",
    "price_type": "tax_exclusive",
    "date": 1574863926,
    "due_date": 1574863926,
    "net_term_days": 0,
    "exchange_rate": 1.0,
    "total": 25200,
    "amount_paid": 25200,
    "amount_adjusted": 0,
    "write_off_amount": 0,
    "credits_applied": 0,
    "amount_due": 0,
    "paid_at": 1574863926,
    "updated_at": 1574863934,
    "resource_version": 1574863934513,
    "deleted": false,
    "object": "invoice",
    "first_invoice": true,
    "amount_to_collect": 0,
    "round_off_amount": 0,
    "new_sales_amount": 25200,
    "has_advance_charges": false,
    "currency_code": "USD",
    "base_currency_code": "USD",
    "generated_at": 1574863926,
    "is_gifted": false,
    "term_finalized": true,
    "channel": "web",
    "tax": 0,
    "line_items": [
        {
            "id": "li_AzyWD9Rj28XT63ng",
            "date_f

In [5]:
print("Ok, ya esta!")



Hola Jim!
